# 6. Output datasets for visualisation

This is a simple utility notebook that gets the data together for visualisation in Flourish.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
df_eric = pd.read_csv("./data/output/trust_mrc_and_epr.csv")
df_eric.head()

,trust_code,trust_name,mrc_off_201718,mrc_off_201819,mrc_off_201920,mrc_off_202021,mrc_off_202122,mrc_off_202223,mrc_off_202324,mrc_on_201718,mrc_on_201819,mrc_on_201920,mrc_on_202021,mrc_on_202122,mrc_on_202223,mrc_on_202324,mrc_service_provision_201718,mrc_service_provision_201819,mrc_service_provision_201920,mrc_service_provision_202021,mrc_service_provision_202122,mrc_service_provision_202223,mrc_service_provision_202324,mrc_type_201718,mrc_type_201819,mrc_type_201920,mrc_type_202021,mrc_type_202122,mrc_type_202223,mrc_type_202324,mrc_total_201718,mrc_total_201819,mrc_total_201920,mrc_total_202021,mrc_total_202122,mrc_total_202223,mrc_total_202324,current_name_in_foi,trust_type_from_foi,trust_epr,in_foi_data,all_admissions_22,emergency_admissions_22,mrc_total_202324_perpatient
0,RTQ,"2GETHER NHS FOUNDATION TRUST, GLOUCESTERSHIRE HEALTH AND CARE NHS FOUNDATION TRUST",34648.0,39826.0,39031.0,111500.0,111500.0,42500.0,64799.0,0.0,0.0,691640.0,115638.0,520733.0,545988.0,547365.0,Internal,Internal,Hybrid,Internal,Internal,Hybrid,Hybrid,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,34648.0,39826.0,730671.0,227138.0,632233.0,588488.0,612164.0,GLOUCESTERSHIRE HEALTH AND CARE NHS FOUNDATION TRUST,Mental Health and Community (Combined),TPP,True,NaN,NaN,NaN
1,REM,"AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST, LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST",65590.0,56657.0,58981.0,101922.0,1646581.0,1534829.0,1120134.0,69250.0,75888.0,78165.0,1435012.0,1646581.0,144750.0,0.0,Other,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,134840.0,132545.0,137146.0,1536934.0,3293162.0,1679579.0,1120134.0,LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST,Acute,Dedalus Healthcare Ltd,True,705871.0,700130.0,1.59
2,RCF,AIREDALE NHS FOUNDATION TRUST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,498051.0,509991.0,553537.0,559974.0,554805.0,577884.0,715884.0,Internal,Internal,Internal,Internal,Internal,Internal,Internal,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,498051.0,509991.0,553537.0,559974.0,554805.0,577884.0,715884.0,AIREDALE NHS FOUNDATION TRUST,Acute,TPP,True,188607.0,195373.0,3.80
3,RBS,ALDER HEY CHILDRENS NHS FOUNDATION TRUST,0.0,70.0,70.0,70.0,187744.0,548176.0,95580.0,71500.0,908371.0,908371.0,908371.0,405918.0,428883.0,438695.0,Internal,Internal,Hybrid,Hybrid,Hybrid,Hybrid,Internal,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,71500.0,908441.0,908441.0,908441.0,593662.0,977059.0,534275.0,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,Acute,Meditech Group Limited,True,NaN,NaN,NaN
4,RTK,ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATION TRUST,53010.0,53010.0,78692.0,92854.0,77554.0,172013.0,74413.0,1001766.0,1001766.0,1076422.0,732069.0,822627.0,804967.0,690551.0,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,1054776.0,1054776.0,1155114.0,824923.0,900181.0,976980.0,764964.0,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST,Acute,Oracle Cerner Corporation,True,351397.0,351397.0,2.18


Give columns readable names.

In [6]:
df_eric['Name'] = df_eric.current_name_in_foi + ' (' + df_eric.trust_code + ")"
df_eric['Trust type'] = df_eric.trust_type_from_foi
df_eric['EPR system'] = df_eric.trust_epr
df_eric['Mean spend per patient'] = df_eric.mrc_total_202324_perpatient #.apply(lambda x: f'£{x:.2f}' if pd.notna(x) else '')
df_eric['Offsite storage costs 2023/24'] = df_eric.mrc_off_202324
df_eric['Onsite storage costs 2023/24'] = df_eric.mrc_on_202324
df_eric['Total spending 2023/24'] = df_eric.mrc_total_202324
df_eric.sort_values(by="Name", inplace=True)

Output just the columns we need and give them pretty names.

Only include the trusts that are in the FOI data - we assume that the rest are not currently active.

In [7]:
data_to_include = df_eric[df_eric.in_foi_data].sort_values(by='Name')
len(df_eric), len(data_to_include)
cols = [
    "Name",
    "Trust type",
    "Total spending 2023/24",
    "Mean spend per patient",
    "Offsite storage costs 2023/24",
    "Onsite storage costs 2023/24",
    "mrc_total_201718", "mrc_total_201819", "mrc_total_201920", "mrc_total_202021",
    "mrc_total_202122", "mrc_total_202223",  "mrc_total_202324",
]
data_to_include[cols].to_csv('./data/output/eric_for_flourish_table.csv', index=False)

## Join with location data, for map

Richard has created a data file from the NHS ODS datasets with latlngs, based on the [NHS postcode directory](https://www.datadictionary.nhs.uk/supporting_information/nhs_postcode_directory.html?hl=nhspd). Join this with our data.

In [8]:
df_locations = pd.read_csv("./data/source/ods/icb_all_loc_england.csv")
df_locations['address'] = df_locations[['AddrLn1', 'AddrLn2', 'AddrLn3', 'Town', 'PostCode']].apply(
    lambda x: ', '.join(str(val).strip() for val in x if pd.notna(val) and str(val).strip() != '')
    , axis=1
)
df_locations.head()

,ICB,OrganisationId,RoleId,RoleName,RelationshipId,RelationshipName,Name,AddrLn1,AddrLn2,AddrLn3,Town,County,PostCode,UPRN,oseast1m,osnrth1m,lat,long,loc_source,loc_type,address
0,QHM,00L,RO319 + RO98,CLINICAL COMMISSIONING GROUP + SUB ICB LOCATION,RE5,IS LOCATED IN THE GEOGRAPHY OF:QHM,NHS NORTH EAST AND NORTH CUMBRIA ICB - 00L,THE RIVERGREEN CENTRE,ST. MARY LANE,ST. MARY PARK,MORPETH,NaN,NE61 6BL,1.009103e+10,418155.0,581031.0,55.123333,-1.716861,NHSPD,P,"THE RIVERGREEN CENTRE, ST. MARY LANE, ST. MARY PARK, MORPETH, NE61 6BL"
1,QHM,00N,RO319 + RO98,CLINICAL COMMISSIONING GROUP + SUB ICB LOCATION,RE5,IS LOCATED IN THE GEOGRAPHY OF:QHM,NHS NORTH EAST AND NORTH CUMBRIA ICB - 00N,MONKTON HALL,MONKTON LANE,MONKTON VILLAGE,JARROW,NaN,NE32 5NN,1.000003e+11,432028.0,563669.0,54.966623,-1.501276,NHSPD,P,"MONKTON HALL, MONKTON LANE, MONKTON VILLAGE, JARROW, NE32 5NN"
2,QHM,00P,RO319 + RO98,CLINICAL COMMISSIONING GROUP + SUB ICB LOCATION,RE5,IS LOCATED IN THE GEOGRAPHY OF:QHM,NHS NORTH EAST AND NORTH CUMBRIA ICB - 00P,PEMBERTON HOUSE,COLIMA AVENUE,SUNDERLAND ENTERPRISE PARK,SUNDERLAND,NaN,SR5 3XB,4.512718e+07,436097.0,557926.0,54.914741,-1.438443,NHSPD,P,"PEMBERTON HOUSE, COLIMA AVENUE, SUNDERLAND ENTERPRISE PARK, SUNDERLAND, SR5 3XB"
3,QHM,01H,RO319 + RO98,CLINICAL COMMISSIONING GROUP + SUB ICB LOCATION,RE5,IS LOCATED IN THE GEOGRAPHY OF:QHM,NHS NORTH EAST AND NORTH CUMBRIA ICB - 01H,4 WAVELL DRIVE,ROSEHILL INDUSTRIAL ESTATE,NaN,CARLISLE,NaN,CA1 2SE,1.000868e+10,342651.0,555644.0,54.892227,-2.895638,NHSPD,P,"4 WAVELL DRIVE, ROSEHILL INDUSTRIAL ESTATE, CARLISLE, CA1 2SE"
4,QHM,102,RO141 + RO33,COUNTY COUNCIL + LOCAL AUTHORITY,RE5,IS LOCATED IN THE GEOGRAPHY OF:QHM,CUMBRIA COUNTY COUNCIL,CUMBRIA HOUSE,117 BOTCHERGATE,NaN,CARLISLE,NaN,CA1 1RD,1.000871e+10,340550.0,555481.0,54.890517,-2.928355,NHSPD,P,"CUMBRIA HOUSE, 117 BOTCHERGATE, CARLISLE, CA1 1RD"


In [9]:
cols = ['OrganisationId', 'lat', 'long', 'address']
df_locations_unique = df_locations[cols].drop_duplicates('OrganisationId', keep='first')
df_with_locations = pd.merge(df_eric, df_locations_unique, left_on="trust_code", right_on="OrganisationId", how="left")
df_with_locations.head()
print(len(df_eric), len(df_with_locations))
df_with_locations.head()

234 234


,trust_code,trust_name,mrc_off_201718,mrc_off_201819,mrc_off_201920,mrc_off_202021,mrc_off_202122,mrc_off_202223,mrc_off_202324,mrc_on_201718,mrc_on_201819,mrc_on_201920,mrc_on_202021,mrc_on_202122,mrc_on_202223,mrc_on_202324,mrc_service_provision_201718,mrc_service_provision_201819,mrc_service_provision_201920,mrc_service_provision_202021,mrc_service_provision_202122,mrc_service_provision_202223,mrc_service_provision_202324,mrc_type_201718,mrc_type_201819,mrc_type_201920,mrc_type_202021,mrc_type_202122,mrc_type_202223,mrc_type_202324,mrc_total_201718,mrc_total_201819,mrc_total_201920,mrc_total_202021,mrc_total_202122,mrc_total_202223,mrc_total_202324,current_name_in_foi,trust_type_from_foi,trust_epr,in_foi_data,all_admissions_22,emergency_admissions_22,mrc_total_202324_perpatient,Name,Trust type,EPR system,Mean spend per patient,Offsite storage costs 2023/24,Onsite storage costs 2023/24,Total spending 2023/24,OrganisationId,lat,long,address
0,RCF,AIREDALE NHS FOUNDATION TRUST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,498051.0,509991.0,553537.0,559974.0,554805.0,577884.0,715884.0,Internal,Internal,Internal,Internal,Internal,Internal,Internal,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,498051.0,509991.0,553537.0,559974.0,554805.0,577884.0,715884.0,AIREDALE NHS FOUNDATION TRUST,Acute,TPP,True,188607.0,195373.0,3.80,AIREDALE NHS FOUNDATION TRUST (RCF),Acute,TPP,3.80,0.0,715884.0,715884.0,RCF,53.897943,-1.962725,"AIREDALE GENERAL HOSPITAL, SKIPTON ROAD, STEETON, KEIGHLEY, BD20 6TD"
1,RBS,ALDER HEY CHILDRENS NHS FOUNDATION TRUST,0.0,70.0,70.0,70.0,187744.0,548176.0,95580.0,71500.0,908371.0,908371.0,908371.0,405918.0,428883.0,438695.0,Internal,Internal,Hybrid,Hybrid,Hybrid,Hybrid,Internal,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,71500.0,908441.0,908441.0,908441.0,593662.0,977059.0,534275.0,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,Acute,Meditech Group Limited,True,NaN,NaN,NaN,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST (RBS),Acute,Meditech Group Limited,NaN,95580.0,438695.0,534275.0,RBS,53.421247,-2.897796,"ALDER HEY HOSPITAL, EATON ROAD, WEST DERBY, LIVERPOOL, L12 2AP"
2,RTK,ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATION TRUST,53010.0,53010.0,78692.0,92854.0,77554.0,172013.0,74413.0,1001766.0,1001766.0,1076422.0,732069.0,822627.0,804967.0,690551.0,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Hybrid,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,1054776.0,1054776.0,1155114.0,824923.0,900181.0,976980.0,764964.0,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST,Acute,Oracle Cerner Corporation,True,351397.0,351397.0,2.18,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST (RTK),Acute,Oracle Cerner Corporation,2.18,74413.0,690551.0,764964.0,RTK,51.377832,-0.527060,"ST PETERS HOSPITAL, GUILDFORD ROAD, CHERTSEY, KT16 0PZ"
3,RVN,AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NHS TRUST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160455.0,177070.0,175827.0,173573.0,175498.0,198135.0,229350.0,Internal,Internal,Internal,Internal,Internal,Internal,Internal,Paper,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,160455.0,177070.0,175827.0,173573.0,175498.0,198135.0,229350.0,AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NHS TRUST,Mental Health and Community (Combined),The Access Group,True,NaN,NaN,NaN,AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NHS TRUST (RVN),Mental Health and Community (Combined),The Access Group,NaN,0.0,229350.0,229350.0,RVN,51.388604,-2.392313,"BATH NHS HOUSE, NEWBRIDGE HILL, BATH, BA1 3QE"
4,RF4,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST",250000.0,194583.0,199237.0,260574.0,394675.0,226800.0,226800.0,1331000.0,1536052.0,1587685.0,1582605.0,1566091.0,1744946.0,1744946.0,Internal,Internal,Internal,Internal,Internal,Internal,Internal,Mixed,1. Paper,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,1581000.0,1730635.0,1786922.0,1843179.0,1960766.0,1971746.0,1971746.0,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST",Acute,System C Healthcare Ltd,True,632011.0,702767.0,3.12,"B

In [10]:
cols = [
    "Name",
    "Trust type",
    "Total spending 2023/24",
    "Spend per patient (where known)",
    "Offsite storage costs 2023/24",
    "Onsite storage costs 2023/24",
    "mrc_total_201718", "mrc_total_201819", "mrc_total_201920", "mrc_total_202021",
    "mrc_total_202122", "mrc_total_202223",  "mrc_total_202324",
    "EPR system",
    "address", "lat", "long"
]
df_with_locations["Spend per patient (where known)"] = df_with_locations['mrc_total_202324_perpatient'].apply(lambda x: f"£{x:,.2f}" if pd.notna(x) else '-')
df_with_locations['Total spending 2023/24'] = df_with_locations["Total spending 2023/24"].astype(int).apply(lambda x: f"£{x:,}" )

# Again, only include the trusts listed in the FOI data.
data_to_include = df_with_locations[df_with_locations.in_foi_data]
data_to_include[cols].to_csv('./data/output/eric_for_flourish_map.csv', index=False)

### Output data for bar chart of total spending

In [11]:
mrc_off_columns = [col for col in df_eric.columns if col.startswith('mrc_off')]
years = [col.split('_')[-1] for col in mrc_off_columns]
result = {
    'Year': [f"{year[:4]}/{year[4:]}" for year in years],
    'Onsite spend': [df_eric[f'mrc_on_{year}'].sum() for year in years],
    'Offsite spend': [df_eric[f'mrc_off_{year}'].sum() for year in years]
}
result_df = pd.DataFrame(result)
print(result_df)
result_df.to_csv('./data/output/total_for_flourish_chart.csv', index=False)

      Year  Onsite spend  Offsite spend
0  2017/18   214650637.0     49687837.0
1  2018/19   181135992.0     50355796.0
2  2019/20   183763447.0     51728441.0
3  2020/21   173666088.0     51684211.0
4  2021/22   175308637.0     58941942.0
5  2022/23   170928754.0     66773505.0
6  2023/24   178363050.0     66292279.0
